# Instalação Pyomo

In [ ]:
#!pip install -q pyomo
!pip install pyomo
!pip install openpyxl
!pip install xlrd

In [ ]:
!pip show pyomo


In [ ]:
import os # Manipular caminhos de arquivos, criar/deletar diretórios
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp
import pyomo.environ as pyo
#from scipy.interpolate import interp1d
from pyomo.opt import SolverFactory
from pyomo.dae import ContinuousSet, DerivativeVar, Integral
from pyomo.environ import NonNegativeReals, exp, TransformationFactory

# Instalação IPOPT

In [ ]:
from pyomo.environ import *
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

In [ ]:
!apt-get remove --purge -y coinor-libipopt-dev
!apt-get install -y coinor-libipopt-dev

In [ ]:
# Atualizar a biblioteca libstdc++6 (necessária para resolver dependências de versão)
!apt-get install -y libstdc++6

In [ ]:
# Executar o comando para encontrar o executável do IPOPT
ipopt_path = !find / -name "ipopt" 2>/dev/null

# Verificar se o comando encontrou algo
if ipopt_path:
    print(f"O executável do IPOPT foi encontrado em: {ipopt_path[0]}")
else:
    print("O comando 'find' não encontrou o executável do IPOPT.")

#Resultado deu diretório no Colab - /content/ipopt

In [ ]:
# Importar o solver IPOPT
solver = SolverFactory('ipopt')

# Verificar as opções padrão do solver
print(solver.options)

# Definir e configurar opções específicas do solver
solver.options['tol'] = 1e-6  # Definir a tolerância do solver
solver.options['max_iter'] = 50000  # Definir o número máximo de iterações
solver.options['print_level'] = 3  # Definir o nível de impressão de saída

In [ ]:
print(solver.options)

# Importando as funções de custo marginais

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pickle

# Função custo marginal não é dependente de (t).
# Carregar o dicionário 'resultados' do arquivo .pkl no Google Drive
with open('/content/drive/MyDrive/funções_outubro/formulas_combinadas.pkl', 'rb') as f:
    resultados = pickle.load(f)

# Verificar as chaves para confirmar que os dados foram carregados corretamente
print(resultados.keys())

In [ ]:
# Função para imprimir as fórmulas
def imprimir_fórmulas(resultados):
    for setor, formula in resultados.items():
        print(f"\nFórmula simbólica para o setor {setor}: {formula}")

# Chamar a função para imprimir as fórmulas
imprimir_fórmulas(resultados)

# Definindo o modelo

In [ ]:
# Definir o modelo
model = pyo.ConcreteModel()

# Bloco 1: Definindo variáveis

In [ ]:
# Carregar os dados da planilha Excel
excel_file = '/content/drive/MyDrive/CÓDIGO DEZEMBRO/TESTE SEM PICO Linear_21_11 - Copia.xlsx'
df = pd.read_excel(excel_file, sheet_name='Planilha1')

# Definir 'Setores' como índice
df = df.set_index('Setores')

# Certifique-se de que os nomes dos setores não tenham espaços adicionais
df.index = [setor.strip() for setor in df.index]

# Verificar a estrutura do DataFrame após limpar os nomes dos setores
print("Setores após remoção de espaços:", df.index)

# Definir os setores como um conjunto em Pyomo
model.setores = pyo.Set(initialize=df.index)


In [ ]:
# Criar o dicionário 'dados_l_i0' a partir do DataFrame transformado
dados_l_i0 = df.T.to_dict()

# Verificar o conteúdo do dicionário 'dados_l_i0'
print("Conteúdo de dados_l_i0:")
for setor, valores in dados_l_i0.items():
    print(f"Setor: {setor}, Valores: {valores}")

In [ ]:
## ------------- VERIFICAÇÃO ---------------##

# Carregar os dados da planilha Excel
excel_file = '/content/drive/MyDrive/CÓDIGO DEZEMBRO/TESTE SEM PICO Linear_21_11 - Copia.xlsx'
df = pd.read_excel(excel_file, sheet_name='Planilha1')

# Exibir as primeiras linhas do DataFrame para verificar a formatação
print("Primeiras linhas do DataFrame:")
print(df.head())

# Verificar se há valores ausentes
print("\nVerificando valores ausentes:")
print(df.isnull().sum())  # Exibe a soma de valores ausentes por coluna

# Verificar tipos de dados das colunas
print("\nTipos de dados das colunas:")
print(df.dtypes)

# Certificar-se de que as colunas de tempo estão no formato correto
# Agora identificar as colunas de tempo como as que são do tipo 'int' (não mais usando 'startswith')
tempos = [col for col in df.columns if isinstance(col, int)]
print("\nColunas de tempo identificadas:", tempos)

# Verificar se as colunas de tempo têm valores numéricos (garantir que são inteiros ou floats)
for col in tempos:
    if not pd.api.types.is_numeric_dtype(df[col]):
        print(f"A coluna {col} não é numérica.")
    else:
        print(f"A coluna {col} é numérica e contém valores do tipo {df[col].dtype}.")

# Bloco 1: Definindo conjuntos

In [ ]:
# CONJUNTOS DE TEMPOS

# Extrair os tempos dos nomes das colunas e definir o conjunto de tempo contínuo
# Definir 'tempos' como uma lista das colunas do DataFrame que já são números inteiros
tempos = [col for col in df.columns if isinstance(col, int)]

# Verificar os tipos dos elementos de 'tempos' para garantir que a conversão foi bem-sucedida
print("Tempos identificados:", tempos)
print("Tipos dos valores em 'tempos':", [type(t) for t in tempos])

# Verificar se todos os valores em 'tempos' são inteiros
if all(isinstance(t, int) for t in tempos):
    # Definir o conjunto de tempo contínuo com os nós explícitos em 'tempos'
    model.time = ContinuousSet(bounds=(min(tempos), max(tempos)), initialize=tempos)
    print(f"Conjunto de tempo contínuo com limites: ({min(tempos)}, {max(tempos)}) e nós: {tempos}")
else:
    print("Erro: O conjunto 'tempos' contém valores não inteiros e não pode ser usado para definir 'model.time'.")


In [ ]:
# CONJUNTOS SETORES

# Definir o conjunto setores como o conjunto de nomes lidos da planilha

# Inicializar o conjunto 'setores' com base nas chaves do dicionário 'dados_l_i0'
model.setores = pyo.Set(initialize=dados_l_i0.keys())
print("Setores no modelo:", list(model.setores))


# Bloco 1: Definindo parâmetros

In [ ]:
## DEFININDO PARÂMETRO l_i0

# Agora, o parâmetro l_i0 é inicializado como um valor constante para cada faixa de tempo.
# Tive de fazer isso, pois percebi que na v1 o tempo era contínuo, integração da função objetivo contínua ao longo do tempo,
# mas os dados de l_i0 eram discretos. Essa opção é mais próxima da realidade de um programa do que tornar o número de licenças contínuo.

# Função degrau para definir l_i0 como constante dentro dos períodos de tempo definidos
def l_i0_step_function(model, setor, t):
    # Obter os tempos conhecidos diretamente como inteiros
    tempos_setor = [col for col in dados_l_i0[setor].keys() if isinstance(col, int)]
    tempos_setor.sort()  # Ordenar tempos para garantir a ordem correta

    # Verificar se t é exatamente igual a um dos tempos conhecidos
    if t in tempos_setor:
        return dados_l_i0[setor][t]

    # Verificar em qual intervalo de tempo o 't' se encontra, agora corrigido para valores intermediários
    for i in range(len(tempos_setor) - 1):
        if tempos_setor[i] <= t < tempos_setor[i + 1]:  # Usar < para garantir que o intervalo seja capturado corretamente
            return dados_l_i0[setor][tempos_setor[i]]

    # Se 't' for maior que o último valor conhecido, retorna o último valor
    return dados_l_i0[setor][tempos_setor[-1]]

# Definir l_i0 como uma expressão que usa a função degrau
model.l_i0_expr = pyo.Expression(model.setores, model.time, rule=l_i0_step_function)

# Verificar a expressão para garantir que tudo está funcionando corretamente
print("\n--- Impressão dos valores de l_i0 para todos os tempos e setores ---")
for setor in model.setores:
    for t in model.time:
        try:
            valor_l_i0 = pyo.value(model.l_i0_expr[setor, t])
            print(f"l_i0_expr[{setor}, {t}] = {valor_l_i0}")
        except KeyError as e:
            print(f"Erro ao acessar l_i0_expr[{setor}, {t}]: {e}")
        except Exception as e:
            print(f"Outro erro ao acessar l_i0_expr[{setor}, {t}]: {e}")





In [ ]:
# Verificar todos os valores de l_i0 antes da transformação DAE
for t in model.time:
    for setor in model.setores:
        print(f"l_i0[{setor}, {t}] = {model.l_i0_expr[setor, t]}")

In [ ]:
## DEFININDO PARÂMETRO L_0

# Definir o valor total de L_0 como função do tempo
def L_0_init(model, t):
    # Usar model.l_i0_expr em vez de model.l_i0 e pyo.value para extrair o valor da Expression
    soma_l_i0 = sum(pyo.value(model.l_i0_expr[i, t]) for i in model.setores)
    return soma_l_i0  # Retorna a soma como L_0 para o tempo 't'

# Definir L_0 como um parâmetro usando a função corrigida
model.L_0 = pyo.Param(model.time, initialize=L_0_init, mutable=True)

# Verificação da soma de l_i0 usando a Expression l_i0_expr
def verificar_soma_l_i0(model, t):
    soma = sum(pyo.value(model.l_i0_expr[i, t]) for i in model.setores)
    if abs(soma - pyo.value(model.L_0[t])) < 1e-6:
        print(f"A soma de l_i0 no tempo {t} é igual a L_0.")
    else:
        print(f"A soma de l_i0 no tempo {t} ({soma}) não é igual a L_0 ({pyo.value(model.L_0[t])}).")

# Verificar a soma de l_i0 em um dado tempo
verificar_soma_l_i0(model, 0)
verificar_soma_l_i0(model, 1)

# Exibir os parâmetros
model.l_i0_expr.pprint()
model.L_0.pprint()

In [ ]:
# Definir uma lista de tempos para visualização
tempos_visuais = list(np.linspace(min(tempos), max(tempos), 100))  # 100 pontos ao longo do intervalo de tempo

# Encontrar os pontos mais próximos em 'model.time' para cada valor de 'tempos_visuais'
tempos_model = np.array(list(model.time))  # Converter model.time para uma lista de pontos
valores_L_0 = []

for t in tempos_visuais:
    # Encontrar o ponto mais próximo em 'model.time' para o valor 't'
    t_proximo = tempos_model[np.abs(tempos_model - t).argmin()]
    valores_L_0.append(pyo.value(model.L_0[t_proximo]))

# Plotar o comportamento de L_0 ao longo do tempo
plt.plot(tempos_visuais, valores_L_0, marker='o')
plt.xlabel('Tempo')
plt.ylabel('L_0')
plt.title('Comportamento de L_0 ao longo do tempo')
plt.grid(True)
plt.show()

# Verificar e imprimir valores de l_i0_expr e L_0
for setor in model.setores:
    for t in model.time:
        print(f"l_i0_expr[{setor}, {t}]: {pyo.value(model.l_i0_expr[setor, t])}")
        print(f"L_0[{t}]: {pyo.value(model.L_0[t])}")


In [ ]:
r = 0.10  # Taxa de desconto
model.r = pyo.Param(initialize=r)  # Taxa de desconto

# Bloco 1: Definindo variáveis

In [ ]:
# VARIÁVEIS DE CONTROLE DOS SETORES
# Variável de ajuste, de fato, o cap é sobre a emissão

model.e_i = pyo.Var(model.setores, model.time, bounds=(0.00001, None))  # Evita zero exato
model.l_i = pyo.Var(model.setores, model.time, bounds=(0.00001, None))  # Evita zero exato

model.p = pyo.Var(model.time, bounds=(0.0001, 100))  # Evita zero exato para o preço


In [ ]:
# VARIÁVEL DE ESTADO DOS SETORES

# a variável de estado depende da variável de controle, e as decisões sobre a variável de controle e_i(t)
# estoque de permissões é influenciado pelas decisões tomadas sobre e_i(t) e segue a dinâmica
# especificada pela equação diferencial.

# Definir a variável de estado B_i
model.B_i = pyo.Var(model.setores, model.time, within=pyo.NonNegativeReals)  # Estoque de emissões bancadas (não pode ser negativo)

# Fixar a condição inicial para B_i no tempo inicial (t = 0)
for setor in model.setores:
    model.B_i[setor, model.time.first()].fix(0)  # B_i(0) = 0 para todos os setores

# Definir a derivada de B_i em relação ao tempo
model.dB_idt = DerivativeVar(model.B_i, wrt=model.time)  # Taxa de variação de B_i

In [ ]:
# VARIÁVEIS DE CONTROLE DO REGULADOR

# Valor total de E_i como função do tempo

# Use model.E como uma variável de controle no problema de otimização.
# Relacionar model.E com a soma de model.e_i através de uma lógica de restrição, mas sem forçar
# a restrição de forma que ela atue como uma soft constraint.

# Definir a variável agregada E para representar a soma das emissões ao longo do tempo
model.E = pyo.Var(model.time, within=pyo.NonNegativeReals)  # Variável agregada de emissões

# Criar uma restrição para associar E à soma das emissões setoriais e_i
def associar_emissoes_aggregate(model, t):
    return model.E[t] == sum(model.e_i[i, t] for i in model.setores)

# Adicionar a restrição de consistência para cada instante de tempo
model.associar_emissoes = pyo.Constraint(model.time, rule=associar_emissoes_aggregate)

# Adicionar uma função objetivo neutra (ou de placeholder) para garantir que o solver funcione
# Isso é apenas para permitir a execução do modelo sem violar a estrutura de Pyomo.
#model.obj = pyo.Objective(expr=0, sense=pyo.minimize)

In [ ]:
# VARIÁVEL DE ESTADO DO REGULADOR

# Definir a soma de todos os estoques B_i como uma expressão para o estoque total B_total em cada instante de tempo
def soma_B_total_rule(model, t):
    return sum(model.B_i[i, t] for i in model.setores)
model.B_total = pyo.Expression(model.time, rule=soma_B_total_rule) # Criar a expressão para o estoque total B_total em cada instante de tempo

# Definir uma variável de estoque total para usar com DerivativeVar
model.B_total_var = pyo.Var(model.time, within=pyo.NonNegativeReals)

# Fixar a condição inicial para B_total_var no tempo inicial (t = 0)
model.B_total_var[model.time.first()].fix(0)  # Define B_total_var(0) = 0

# Definir a derivada do estoque total B_total_var em relação ao tempo
model.dB_total_dt = DerivativeVar(model.B_total_var, wrt=model.time)

# Criar uma restrição para garantir que model.B_total_var seja igual a model.B_total em cada instante de tempo
def consistency_rule(model, t):
    return model.B_total_var[t] == model.B_total[t]

# Adicionar a restrição de consistência ao modelo
model.consistency_constraint = pyo.Constraint(model.time, rule=consistency_rule)


# Bloco 2: Inicialização

In [ ]:
def initialize_variables(model):
    print("Inicializando variáveis...")  # Verificação inicial

    #valor_manual_E = 0  # Inicializar a variável para armazenar a soma

    # Inicializar variáveis para t=0
    for setor in model.setores:
        for t in model.time:
            if t == model.time.first():  # Fixar apenas para t=0
                if setor == 'Celulose':
                    valor_inicial_e_i = 5901419 / 1000
                elif setor == 'Cimento':
                    valor_inicial_e_i = 34224339 / 1000
                elif setor == 'VidCerm':
                    valor_inicial_e_i = 4561417 / 1000
                elif setor == 'FerroGussa':
                    valor_inicial_e_i = 51117070 / 1000
                elif setor == 'ProdutosMet':
                    valor_inicial_e_i = 11361545 / 1000
                elif setor == 'Saneamento':
                    valor_inicial_e_i = 85744711 / 1000
                elif setor == 'PetroGas':
                    valor_inicial_e_i = 25821432 / 1000
                elif setor == 'UTC':
                    valor_inicial_e_i = 7541320/ 1000
                elif setor == 'UTG':
                    valor_inicial_e_i = 12172604/ 1000
                elif setor == 'UTOD':
                    valor_inicial_e_i = 2425250/ 1000
                elif setor == 'UTOC':
                    valor_inicial_e_i = 30198/ 1000
                elif setor == 'TranspCarga':
                    valor_inicial_e_i = 114937896/ 1000
                elif setor == 'TransPass':
                    valor_inicial_e_i = 92843424/ 1000
                elif setor == 'TransAere':
                    valor_inicial_e_i = 9549379/ 1000

                # Fixar o valor de e_i[setor, t] para t=0
                model.e_i[setor, t].fix(valor_inicial_e_i)
                print(f"Inicializando e_i[{setor}, {t}] com valor {valor_inicial_e_i} para t=0")

                model.dB_idt[setor, t].fix(0)
                print(f"fixando dB_idt[{setor}, {t}] para t=0")

    # Inicializar o preço p e outras variáveis globais apenas para t=0
    for t in model.time:
        if t == model.time.first():
            print(f"Inicializando p[{t}] para t=0")
            model.p[t].set_value(3)

    print("Inicialização e verificação concluídas.")

    # Impressão dos valores após inicialização
    for setor in model.setores:
        for t in model.time:
            print(f"Setor: {setor}, Tempo: {t}")
            print(f"e_i[{setor}, {t}] = {model.e_i[setor, t].value}")
            print(f"p[{t}] = {model.p[t].value}")

# Certifique-se de que a função seja chamada corretamente
initialize_variables(model)


In [ ]:
for setor in model.setores:
    for t in model.time:
        if t == 0:
            print(f"e_i[{setor}, {t}] fixado em: {model.e_i[setor, t].fixed}")
            print(f"Valor inicial de e_i[{setor}, {t}]: {model.e_i[setor, t].value}")


# Bloco 3: Discretização

In [ ]:
# Transformação DAE antes de resolver o modelo
# DAE afeta todas as variáveis que dependem do tempo no modelo

TransformationFactory('dae.finite_difference').apply_to(model, nfe=32)

In [ ]:
def verificar_l_i0(model, dados_l_i0):
    print("\nVerificação dos valores de l_i0 após a transformação DAE:\n")
    erros = False

    # Extrair os tempos conhecidos do dicionário diretamente como inteiros
    setores = list(dados_l_i0.keys())
    tempos_setor = [col for col in dados_l_i0[setores[0]].keys() if isinstance(col, int)]
    tempos_setor.sort()  # Ordenar tempos para garantir a ordem correta

    for t in model.time:
        for setor in model.setores:
            # Encontrar o intervalo ao qual o tempo pertence
            for i in range(len(tempos_setor) - 1):
                if tempos_setor[i] <= t < tempos_setor[i + 1]:
                    chave = tempos_setor[i]  # Agora a chave é o número inteiro diretamente
                    valor_esperado = dados_l_i0[setor][chave]
                    break
            else:
                # Se t for igual ou maior ao último tempo do DataFrame
                chave_final = tempos_setor[-1]
                valor_esperado = dados_l_i0[setor][chave_final]

            try:
                valor_modelo = pyo.value(model.l_i0_expr[setor, t])  # Use 'l_i0_expr' para obter o valor correto
            except ValueError:
                valor_modelo = None

            if valor_modelo == valor_esperado:
                print(f"CORRETO: l_i0[{setor}, {t}] = {valor_modelo} (esperado: {valor_esperado})")
            else:
                print(f"ERRO: l_i0[{setor}, {t}] = {valor_modelo} (esperado: {valor_esperado})")
                erros = True

    if not erros:
        print("\nTodos os valores de l_i0 foram atribuídos corretamente!\n")
    else:
        print("\nAlguns valores de l_i0 estão incorretos!\n")

# Chamar a função de verificação após a transformação DAE
verificar_l_i0(model, dados_l_i0)


# Bloco 4: Problema da firma

In [ ]:
 # CRIANDO F_i_FUNCTIONS PARA CADA SETOR

 # SymPy transforma uma expressão simbólica (do código de compilação das regressões polinomiais setorais)
 # em uma função númerica para uso na função de otimização.

# Criar F_i_functions para cada setor, ajustando a variável conforme necessário
F_i_functions = {}
for setor, conteudo in resultados.items():
    # Verificar se 'conteudo' é uma string e convertê-la para expressão simbólica
    if isinstance(conteudo, str):
        formula = sp.sympify(conteudo)  # Converte string para expressão simbólica
    elif isinstance(conteudo, dict) and 'formula_simbolica' in conteudo:
        formula = conteudo['formula_simbolica']
    else:
        print(f"Setor {setor} não possui uma fórmula válida. Verifique os dados.")
        continue

    # Ajustar a variável 'x' para 'e'
    formula = formula.subs('x', sp.Symbol('e'))  # Substitui 'x' por 'e'

    # Imprimir a fórmula simbólica após a substituição
    print(f"Fórmula simbólica ajustada para o setor {setor}: {formula}")

    # Criar a função lambdificada com 'e' como variável
    try:
        F_i_functions[setor] = sp.lambdify('e', formula, 'numpy')
        print(f"Função lambdificada criada para o setor {setor}: {F_i_functions[setor]}")
    except Exception as e:
        print(f"Erro ao lambdificar a fórmula para o setor {setor}: {e}")

# Verificar as funções lambdificadas
for setor, func in F_i_functions.items():
    print(f"Função lambdificada para o setor {setor}: {func}")

In [ ]:
# Adicione a definição de dt ao modelo, se ainda não estiver definido
model.dt = pyo.Param(initialize=0.1, mutable=True)  # Por exemplo, 0.1 como intervalo de tempo. Ajuste conforme necessário.

In [ ]:
## ----------------  Objetivo ----------------- ##

# Função que define o integrando da "integral" para cada setor
def objective_integrand_rule(model, s, t):
    F_i_func = F_i_functions[s]  # Refere-se à função de custo marginal do setor s
    return pyo.exp(-model.r * t) * (F_i_func(model.e_i[s, t]) + model.p[t] * (model.l_i[s, t] - model.l_i0_expr[s, t]))

# Aproximação da integral como somatório ao longo do tempo
def setor_objective_rule(model, s):
    return sum(objective_integrand_rule(model, s, t) for t in model.time) * model.dt

# ---------------- Market Clearing -------------- ##

# Market clearing 1
def market_clearing_1_rule(model, t):
    return sum((model.l_i[setor, t] - model.l_i0_expr[setor, t]) for setor in model.setores) == 0

# Market Clearing 2: Garante que o preço final é zero se houver estoque ou que o estoque
# final é zero se o preço for diferente de zero.
def market_clearing_2_rule(model):
    return model.p[model.time.last()] * sum(model.B_i[setor, model.time.last()] for setor in model.setores) == 0

## ----------------- Restrições ----------------- ##

def emission_limit_rule(model, setor, t):
    return model.e_i[setor, t] <= model.l_i[setor, t]

# Definir a equação dinâmica como uma restrição
def balanco_emissao_rule(model, s, t):
    if t == model.time.first():
        return pyo.Constraint.Skip  # Ignorar a equação dinâmica no tempo inicial
    #return model.dB_idt[s, t] == model.l_i0_expr[s, t] - model.e_i[s, t] + (model.l_i[s, t] - model.l_i0_expr[s, t])#diferenciador_setores[s]
    return  model.dB_idt[s, t] == model.l_i[s, t] - model.e_i[s, t]


## ----- Aplicar Penalidades Progressivas em Intervalos ----- ##

# Penalidade por Intervalo - Penalidade por Intervalo: Introduz um aumento escalonado menos abrupto do que a penalidade cúbica, penalizando gradualmente e incentivando uma gestão balanceada das permissões ao longo do temp
# Incentivo moderado para o consumo ao longo do tempo: Use a penalidade por intervalo.

def penalidade_banking_rule(model, setores): #ESSA FOI A MELHOR
    penalidade = 1100 #Ajuste conforme necessário
    return sum((1 + (t // 2)) * penalidade * model.B_i[setor, t] for setor in model.setores for t in model.time if t >= 8)

# Adiciona a penalidade de banking como uma Expression no modelo
model.penalidade_banking = pyo.Expression(model.setores, rule=penalidade_banking_rule)


# Bloco 5: Problema do regulador

In [ ]:
# Definir as variáveis simbólicas 'e' e 't' no SymPy
e = sp.Symbol('e')
t = sp.Symbol('t')  # Definindo 't' como uma variável simbólica

# Criar dicionário para armazenar as funções marginais simbólicas de cada setor
F_i_symbolic_functions = {}

for setor, conteudo in resultados.items():
    if isinstance(conteudo, str):
        formula = sp.sympify(conteudo)
    elif isinstance(conteudo, dict) and 'formula_simbolica' in conteudo:
        formula = conteudo['formula_simbolica']
    else:
        print(f"Setor {setor} não possui uma fórmula válida. Verifique os dados.")
        continue

    # Ajustar a variável 'x' para 'e' e adicionar dependência de 't' se necessário
    formula = formula.subs('x', e)  # Aqui você pode adicionar substituições relacionadas a 't', se existirem

    # Armazenar a função simbólica no dicionário
    F_i_symbolic_functions[setor] = formula

# Agregar todas as funções marginais setoriais em uma única função agregada, com 't' incluído
F_agregado = sum(F_i_symbolic_functions.values())

# Imprimir e lambdificar a função agregada
F_agregado_func = sp.lambdify((e, t), F_agregado, 'numpy')

In [ ]:
## --------------- Objetivo --------------------- ##

# Função que define o integrando da "integral" para o regulador ao longo do tempo
def objective_regulator_integrand_rule(model, t):
    # Calcular o valor da função agregada para E[t]
    valor_func = F_agregado_func(model.E[t],t)  # Use F_agregado_func
    return pyo.exp(-model.r * t) * valor_func

# Aproximação da integral como somatório ao longo do tempo para o regulador
def regulator_objective_rule(model):
    return sum(objective_regulator_integrand_rule(model, t) for t in model.time) * model.dt

## ----------------- Restrições ----------------- ##

# Restrição de balanço dinâmico para o regulador, conforme seu código original
def balance_rule(model, t):
    return model.dB_total_dt[t] == model.L_0[t] - model.E[t]  # model.E[t] é a soma das emissões agregadas


# Bloco 5: Inicialização

In [ ]:
initialize_variables(model)

In [ ]:
# Verificar a definição e inicialização de l_i0_expr
for i in model.setores:
    for t in model.time:
        try:
            print(f"l_i0_expr[{i}, {t}] = {pyo.value(model.l_i0_expr[i, t])}")
        except ValueError:
            print(f"Erro: l_i0_expr[{i}, {t}] não está inicializado corretamente.")

# Corrigir a inicialização de L_0 com tratamento de erros
def L_0_init(model, t):
    try:
        # Verificar se todos os valores em l_i0_expr são válidos
        soma_l_i0 = sum(pyo.value(model.l_i0_expr[i, t]) for i in model.setores if model.l_i0_expr[i, t] is not None)

        # Verificar se soma_l_i0 é válido
        if soma_l_i0 is None:
            print(f"Erro ao inicializar L_0 em t={t}. Usando valor padrão 0.0")
            return 0.0  # Valor padrão em caso de erro

        return soma_l_i0
    except ValueError:
        print(f"Erro ao inicializar L_0 em t={t}. Usando valor padrão 0.0")
        return 0.0  # Valor padrão em caso de erro

# Definir L_0 como um parâmetro usando a função corrigida
model.L_0 = pyo.Param(model.time, initialize=L_0_init, mutable=True)

# Verificar a inicialização de L_0
for t in model.time:
    print(f"L_0[{t}] = {model.L_0[t]}")

# Verificação da soma de l_i0 usando a Expression l_i0_expr
def verificar_soma_l_i0(model, t):
    try:
        soma = sum(pyo.value(model.l_i0_expr[i, t]) for i in model.setores)
        if abs(soma - pyo.value(model.L_0[t])) < 1e-6:
            print(f"A soma de l_i0 no tempo {t} é igual a L_0.")
        else:
            print(f"A soma de l_i0 no tempo {t} ({soma}) não é igual a L_0 ({pyo.value(model.L_0[t])}).")
    except ValueError:
        print(f"Erro ao verificar a soma de l_i0 no tempo {t}. Algum valor não foi inicializado corretamente.")

# Verificar a soma de l_i0 em um dado tempo
verificar_soma_l_i0(model, 0)
verificar_soma_l_i0(model, 1)

# Exibir os parâmetros
model.l_i0_expr.pprint()
model.L_0.pprint()

# Bloco 6: Otimização

In [ ]:
def resolver_modelo(model):
    solver = pyo.SolverFactory('ipopt')
    solver.set_executable('/content/ipopt', validate=False)

    resultados_calculados = {}

    # Setores: Adiciona as restrições de market clearing globais
    model.market_clearing_1 = pyo.Constraint(model.time, rule=market_clearing_1_rule)
    model.market_clearing_2 = pyo.Constraint(rule=market_clearing_2_rule)

    # Setores: Variável para armazenar explicitamente o Objetivo de Cada Setor (pyo.Var)
    model.objective_individual = pyo.Var(model.setores, within=pyo.Reals)

    # Setores: Regra para Calcular o Valor da Função Objetivo Individual com a penalidade
    def store_objective_individual_rule(model, setor):
        return model.objective_individual[setor] == (setor_objective_rule(model, setor) + model.penalidade_banking[setor]) # Penalidade progressiva ao longo do tempo
            # + model.penalidade_ultimos_periodos [setor]  # Penalidade extra no último período

    # Constraint para garantir o cálculo do objetivo individual
    model.store_objective_individual = pyo.Constraint(model.setores, rule=store_objective_individual_rule)

    # Regulador: Definir a função objetivo do regulador como uma variável pyo.Var
    model.objective_regulator = pyo.Var(within=pyo.Reals)

    # Definir a função objetivo do regulador
    def store_regulator_objective_rule(model):
        return model.objective_regulator == regulator_objective_rule(model)

    model.store_regulator_objective = pyo.Constraint(rule=store_regulator_objective_rule)

    # Definir uma função objetivo global para combinar firmas e regulador
    def objective_rule_global(model):
        return sum(model.objective_individual[setor] for setor in model.setores) + model.objective_regulator

    model.objective_global = pyo.Objective(rule=objective_rule_global, sense=pyo.minimize)

    # Setor: Adiciona as restrições de emissões e dinâmica
    model.balanco_emissao = pyo.Constraint(model.setores, model.time, rule=balanco_emissao_rule)
    model.emission_limit = pyo.Constraint(model.setores, model.time, rule=emission_limit_rule)

    # Regulador: Adicionar a restrição de balanço do regulador
    model.balance = pyo.Constraint(model.time, rule=balance_rule)

    # Resolver o modelo para todos os setores
    result = solver.solve(model, tee=True)

    # Armazenar os resultados calculados para cada setor e cada tempo
    for setor in model.setores:
        # Certifique-se de que cada setor tenha um dicionário para armazenar os valores
        resultados_calculados[setor] = {}  # Criar um dicionário para cada setor
        for t in model.time:
            resultados_calculados[setor][t] = {
                'e_i': pyo.value(model.e_i[setor, t]),
                'l_i': pyo.value(model.l_i[setor, t]),
                'p': pyo.value(model.p[t]),
                'B_i': pyo.value(model.B_i[setor, t]) if hasattr(model, 'B_i') else None,
                'dB_idt': pyo.value(model.dB_idt[setor, t]) if hasattr(model, 'dB_idt') else None,
                'objective_individual': pyo.value(model.objective_individual[setor]),
                'objective_global': pyo.value(model.objective_global)
            }

    # Armazenar os resultados do regulador
    resultados_regulador = {}
    for t in model.time:
        resultados_regulador[t] = {
            'objetivo_regulador': pyo.value(model.objective_regulator),
            'E': pyo.value(model.E[t]) if hasattr(model, 'E') else None,
            'B_total_var': pyo.value(model.B_total_var[t]) if hasattr(model, 'B_total_var') else None,
            'dB_total_dt': pyo.value(model.dB_total_dt[t]) if hasattr(model, 'dB_total_dt') else None
        }

    return resultados_calculados, resultados_regulador

In [ ]:
# Resolver o modelo com IPOPT
solver = SolverFactory('ipopt')
resultado = solver.solve(model, tee=True)

# Verificar o status do solver e a condição de terminação
if resultado.solver.status == pyo.SolverStatus.ok and resultado.solver.termination_condition == pyo.TerminationCondition.optimal:
    print("Solver encontrou uma solução ótima.")
    model.solutions.load_from(resultado)
elif resultado.solver.termination_condition == pyo.TerminationCondition.infeasible:
    print("O problema é inviável.")
elif resultado.solver.termination_condition == pyo.TerminationCondition.unbounded:
    print("O problema é ilimitado.")
else:
    print(f"Solver status: {resultado.solver.status}")
    print(f"Termination condition: {resultado.solver.termination_condition}")


In [ ]:
# ----- Bloco principal ----- #

# Inicializa as variáveis e resolve o modelo
# initialize_variables(model)

# Resolver o modelo com as condições de market clearing
resultados_calculados, resultados_regulador = resolver_modelo(model)  # Desempacotar os dois dicionários

# Exibir os resultados detalhados para cada setor e tempo (resultados das firmas)
for setor, valores in resultados_calculados.items():
    print(f"\nSetor: {setor}")
    for t, resultado in valores.items():
        print(f" - Tempo (t): {t}")
        print(f"   - Emissões (e_i): {resultado['e_i']}")
        print(f"   - Permissões de emissão adquiridas (l_i): {resultado['l_i']}")
        print(f"   - Preço (p): {resultado['p']}")
        if 'B_i' in resultado and resultado['B_i'] is not None:
            print(f"   - Estoque (B_i): {resultado['B_i']}")
        if 'dB_idt' in resultado and resultado['dB_idt'] is not None:
            print(f"   - Derivada do Estoque (dB_idt): {resultado['dB_idt']}")

# Exibir os resultados consolidados de objetivo individual e global por setor (resultados das firmas)
for setor, valores in resultados_calculados.items():
    print(f"\nSetor: {setor}")
    # Função objetivo individual e global para o último tempo, pois são acumulados ao longo do tempo
    ultima_etapa = list(valores.keys())[-1]
    print(f" - Função objetivo individual: {resultados_calculados[setor][ultima_etapa]['objective_individual']}")
    print(f" - Função objetivo global: {resultados_calculados[setor][ultima_etapa]['objective_global']}")

# Impressão dos valores de B_i para cada setor e cada tempo após a resolução do modelo
for setor in model.setores:
    for t in model.time:
      print(f"B_i para setor {setor}, tempo {t}: {pyo.value(model.B_i[setor, t])}")

# Resultados Setores

In [ ]:
from IPython.display import display  # Importar display para exibir df
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
# Formatar os resultados das firmas em um df
def formatar_resultados_firmas(resultados_calculados):

    # Lista para armazenar os dados
    dados_tabela_firmas = []

    # Iterar pelos setores e tempos
    for setor, valores in resultados_calculados.items():
        for t, resultado in valores.items():
            # Adiciona os resultados das firmas em uma estrutura de tabela
            dados_tabela_firmas.append({
                'Setor': setor,
                'Tempo': t,
                'Emissões (e_i)': resultado['e_i'],
                'Permissões de emissão (l_i)': resultado['l_i'],
                'Preço (p)': resultado['p'],
                'Estoque (B_i)': resultado['B_i'] if 'B_i' in resultado else None,
                'Derivada do Estoque (dB_idt)': resultado['dB_idt'] if 'dB_idt' in resultado else None,
                'Função objetivo individual': resultado['objective_individual']
            })

    # Retorna os dados como DataFrame
    return pd.DataFrame(dados_tabela_firmas)

# Exibir os resultados dos setores
def exibir_resultados_firmas(df_resultados_firmas):
    display(df_resultados_firmas)  # Exibir df

    # Exportar para Excel
    nome_arquivo = '/content/drive/My Drive/resultados_exportados_FIRMAS_Linear.xlsx' # Altere o caminho se necessário
    df_resultados_firmas.to_excel(nome_arquivo, index=False)
    print(f"Resultados das firmas exportados")

# Formatar e exibir os resultados das firmas
df_resultados_firmas = formatar_resultados_firmas(resultados_calculados)
exibir_resultados_firmas(df_resultados_firmas)

In [ ]:
def verificar_estrutura_dicionario(dicionario):
    for setor, valores in dicionario.items():
        if not isinstance(valores, dict):
            print(f"Erro: Esperado um dicionário para o setor {setor}, mas foi encontrado {type(valores)}")
        else:
            for t, resultado in valores.items():
                if not isinstance(resultado, dict):
                    print(f"Erro: Esperado um dicionário para o tempo {t} no setor {setor}, mas foi encontrado {type(resultado)}")
                else:
                    print(f"Setor {setor}, Tempo {t}: Dicionário de resultados está correto.")

# Verificar a estrutura de resultados_calculados
verificar_estrutura_dicionario(resultados_calculados)

In [ ]:
# SALVAR EM UM ÚNICO PDF

def plot_graficos_emissoes_permissoes_unico(resultados):
    # Determinar o número de setores para ajustar o layout dos subplots
    num_setores = len(resultados)

    # Criar uma figura grande o suficiente para comportar os gráficos
    fig, axes = plt.subplots(num_setores, 2, figsize=(12, 5 * num_setores))  # 2 colunas (para emissões e permissões)

    # Checar se 'axes' é uma matriz ou uma única instância, caso haja apenas um setor
    if num_setores == 1:
        axes = [axes]

    # Iterar por cada setor
    for i, (setor, valores) in enumerate(resultados.items()):
        # Verificar se 'valores' é um dicionário
        if not isinstance(valores, dict):
            print(f"Erro: Esperado um dicionário para o setor {setor}, mas foi encontrado {type(valores)}")
            continue

        # Listas para armazenar dados para o gráfico
        tempos = []
        emissoes = []
        permissoes = []

        # Preencher listas com dados de emissões e permissões para cada tempo
        for t, resultado in valores.items():
            tempos.append(t)
            emissoes.append(resultado['e_i'])
            permissoes.append(resultado['l_i'])

        # Gráfico de emissões (e_i)
        axes[i][0].plot(tempos, emissoes, marker='o', linestyle='-', color='blue', label='Emissões (e_i)')
        axes[i][0].set_title(f'Emissões ao longo do tempo - Setor: {setor}')
        axes[i][0].set_xlabel('Tempo')
        axes[i][0].set_ylabel('Emissões (e_i)')
        axes[i][0].grid(True)
        axes[i][0].legend()

        # Gráfico de permissões de emissão (l_i)
        axes[i][1].plot(tempos, permissoes, marker='o', linestyle='-', color='green', label='Permissões (l_i)')
        axes[i][1].set_title(f'Permissões de emissão ao longo do tempo - Setor: {setor}')
        axes[i][1].set_xlabel('Tempo')
        axes[i][1].set_ylabel('Permissões (l_i)')
        axes[i][1].grid(True)
        axes[i][1].legend()

    # Ajustar layout dos gráficos
    plt.tight_layout()

    # Salvar todos os gráficos em um único arquivo PDF
    with PdfPages('/content/drive/My Drive/graficos_FIRMAS_emissoes_permissoes_Linear.pdf') as pdf:
        pdf.savefig(fig)

    # Exibir o gráfico
    plt.show()

# Chamar a função para plotar e salvar todos os gráficos em um único arquivo PDF
# Certifique-se de que 'resultados' é um dicionário no formato correto
plot_graficos_emissoes_permissoes_unico(resultados_calculados)


In [ ]:
def plot_graficos_emissoes_permissoes(df_resultados_firmas):
    # Definir o caminho do arquivo PDF
    pdf_filename = "/mnt/data/graficos_emissoes_permissoes.pdf"

    # Criar um arquivo PDF para armazenar os gráficos
    with PdfPages(pdf_filename) as pdf:
        # Obter lista de setores únicos
        setores = df_resultados_firmas['Setor'].unique()

        # Iterar por cada setor
        for setor in setores:
            # Filtrar os dados para o setor atual
            df_setor = df_resultados_firmas[df_resultados_firmas['Setor'] == setor]

            # Criar gráfico
            plt.figure(figsize=(8, 5))
            plt.plot(df_setor['Tempo'], df_setor['Emissões (e_i)'], marker='o', linestyle='-', color='blue', label='Emissões (e_i)')
            plt.plot(df_setor['Tempo'], df_setor['Permissões de emissão (l_i)'], marker='s', linestyle='--', color='green', label='Permissões (l_i)')
            plt.title(f'Emissões e Permissões ao longo do tempo - Setor: {setor}')
            plt.xlabel('Tempo')
            plt.ylabel('Valores')
            plt.grid(True)
            plt.legend()



    print(f"Arquivo PDF salvo em: {pdf_filename}")
    return pdf_filename

# Gerar os gráficos e salvar em um PDF
pdf_filepath = plot_graficos_emissoes_permissoes(df_resultados_firmas)

# Exibir o caminho do arquivo gerado
print(f"Gráficos salvos em: {pdf_filepath}")


In [ ]:
# SALVAR EM UM ÚNICO PDF

def plot_graficos_estoque_derivada_unico(resultados):
    # Determinar o número de setores para ajustar o layout dos subplots
    num_setores = len(resultados)

    # Criar uma figura grande o suficiente para comportar os gráficos
    fig, axes = plt.subplots(num_setores, 2, figsize=(12, 5 * num_setores))  # 2 colunas (para Estoque e Derivada)

    # Checar se 'axes' é uma matriz ou uma única instância, caso haja apenas um setor
    if num_setores == 1:
        axes = [axes]

    # Iterar por cada setor
    for i, (setor, valores) in enumerate(resultados.items()):
        # Listas para armazenar dados para o gráfico
        tempos = []
        estoque = []
        derivada_estoque = []

        # Preencher listas com dados de estoque e derivada do estoque para cada tempo
        for t, resultado in valores.items():
            tempos.append(t)
            estoque.append(resultado['B_i'])
            derivada_estoque.append(resultado['dB_idt'])

        # Gráfico de Estoque (B_i)
        axes[i][0].plot(tempos, estoque, marker='o', linestyle='-', color='blue', label='Estoque (B_i)')
        axes[i][0].set_title(f'Estoque ao longo do tempo - Setor: {setor}')
        axes[i][0].set_xlabel('Tempo')
        axes[i][0].set_ylabel('Estoque (B_i)')
        axes[i][0].grid(True)
        axes[i][0].legend()

        # Gráfico de Derivada do Estoque (dB_idt)
        axes[i][1].plot(tempos, derivada_estoque, marker='o', linestyle='-', color='green', label='Derivada do Estoque (dB_idt)')
        axes[i][1].set_title(f'Derivada do Estoque ao longo do tempo - Setor: {setor}')
        axes[i][1].set_xlabel('Tempo')
        axes[i][1].set_ylabel('Derivada do Estoque (dB_idt)')
        axes[i][1].grid(True)
        axes[i][1].legend()

    # Ajustar layout dos gráficos
    plt.tight_layout()

    # Salvar todos os gráficos em um único arquivo PDF
    with PdfPages('/content/drive/My Drive/graficos_FIRMAS_estoque_Linear.pdf') as pdf:
        pdf.savefig(fig)

    # Exibir o gráfico
    plt.show()

# Chamar a função para plotar e salvar todos os gráficos em um único arquivo PDF
plot_graficos_estoque_derivada_unico(resultados_calculados)


# Resultados Regulador

In [ ]:
# Formatar os resultados do regulador em um df

def formatar_resultados_regulador(resultados_regulador, model):

    # Lista para armazenar os dados
    dados_tabela_regulador = []

    # Iterar pelos tempos para organizar os resultados do regulador
    for t, resultado in resultados_regulador.items():
        dados_tabela_regulador.append({
            'Tempo': t,
            'Emissões agregadas (E[t])': resultado['E'],
            'Estoque total (B_total_var[t])': resultado['B_total_var'],
            'Derivada do estoque total (dB_total_dt[t])': resultado['dB_total_dt'],
            'Objetivo do Regulador': resultado['objetivo_regulador']
        })

    # Retorna os dados como df
    return pd.DataFrame(dados_tabela_regulador)

# Exibir os resultados do regulador
def exibir_resultados_regulador(df_resultados_regulador):
    display(df_resultados_regulador)  # Exibir df

    # Exportar para Excel
    nome_arquivo = '/content/drive/My Drive/resultados_exportados_REGULADOR_Linear.xlsx'  # Altere o caminho se necessário
    df_resultados_regulador.to_excel(nome_arquivo, index=False)
    print(f"Resultados do regulador exportados")

# Formatar e exibir os resultados do regulador
df_resultados_regulador = formatar_resultados_regulador(resultados_regulador, model)
exibir_resultados_regulador(df_resultados_regulador)

# Resultado Consolidado

In [ ]:
# Função para criar uma tabela simplificada com Setor, Função Objetivo Individual e Função Objetivo do Regulador
def criar_tabela_simplificada_com_regulador(resultados_calculados, resultados_regulador):
    # Lista para armazenar os dados
    tabela_simplificada = []

    # Iterar pelos setores
    for setor, valores in resultados_calculados.items():
        # Considerar apenas o último tempo, pois a função objetivo é acumulada
        ultima_etapa = list(valores.keys())[-1]
        tabela_simplificada.append({
            'Setor': setor,
            'Penalidade = 1100 t>8': valores[ultima_etapa]['objective_individual']
        })

    # Adicionar a função objetivo do regulador (último período)
    ultimo_periodo = list(resultados_regulador.keys())[-1]
    tabela_simplificada.append({
        'Setor': 'Regulador',
        'Penalidade = 1100 t>8': resultados_regulador[ultimo_periodo]['objetivo_regulador']  # Ajustado para t>6
    })

    # Converter a lista em DataFrame
    df_tabela = pd.DataFrame(tabela_simplificada)

    # Ajustar o índice para começar em 1
    df_tabela.index = range(1, len(df_tabela) + 1)

    # Criar um cabeçalho multi-nível corrigido
    df_tabela.columns = pd.MultiIndex.from_tuples(
        [('Setor', ''), ('Função Objetivo', 'Penalidade = 1100 t>8')]  # Consistência no cabeçalho
    )

    return df_tabela

# Gerar a nova tabela simplificada com o regulador
df_tabela_simplificada_com_regulador = criar_tabela_simplificada_com_regulador(resultados_calculados, resultados_regulador)

# Exibir ou salvar a tabela simplificada
if not df_tabela_simplificada_com_regulador.empty:
    display(df_tabela_simplificada_com_regulador)
    df_tabela_simplificada_com_regulador.to_excel('/content/drive/My Drive/Penal_Linear_1100_t_8_tabela_consolidado.xlsx', index=True)
    print("Tabela simplificada com regulador exportada para Excel.")
else:
    print("Erro: O DataFrame da tabela simplificada está vazio.")

In [ ]:
# Renomear colunas para acesso mais fácil
df_resultados_regulador = df_resultados_regulador.rename(columns={
    'Tempo': 'tempo',
    'Emiss\u00f5es agregadas (E[t])': 'emissoes',
    'Estoque total (B_total_var[t])': 'estoque_total',
    'Derivada do estoque total (dB_total_dt[t])': 'derivada_estoque',
    'Objetivo do Regulador': 'objetivo_regulador'
})

# Calcular a soma dos valores iniciais dos setores para E[t=0]
valores_iniciais_setores = {
    'Celulose': 5901419 / 1000,
    'Cimento': 34224339 / 1000,
    'VidCerm': 4561417 / 1000,
    'FerroGussa': 51117070 / 1000,
    'ProdutosMet': 11361545 / 1000,
    'Saneamento': 85744711 / 1000,
    'PetroGas': 25821432 / 1000,
    'UTC': 7541320 / 1000,
    'UTG': 12172604 / 1000,
    'UTOD': 2425250 / 1000,
    'UTOC': 30198 / 1000,
    'TranspCarga': 114937896 / 1000,
    'TransPass': 92843424 / 1000,
    'TransAere': 9549379 / 1000,
}

# Soma dos valores iniciais
E_t0 = sum(valores_iniciais_setores.values())
print(f"E[t=0]: {E_t0}")

# Substituir E[t=0] no DataFrame para garantir consistência
df_resultados_regulador.loc[df_resultados_regulador['tempo'] == 0, 'emissoes'] = E_t0

# Verificar a existência de colunas adicionais antes de plotar
if 'custo_total' in df_resultados_regulador.columns:
    plt.plot(df_resultados_regulador['tempo'], df_resultados_regulador['custo_total'], label='Custo Total', marker='o')

if 'preco_mercado' in df_resultados_regulador.columns:
    plt.plot(df_resultados_regulador['tempo'], df_resultados_regulador['preco_mercado'], label='Preço de Mercado', marker='^')

# Plotar variáveis principais
plt.plot(df_resultados_regulador['tempo'], df_resultados_regulador['emissoes'], label='Emissões Agregadas (E[t])', marker='s')
plt.plot(df_resultados_regulador['tempo'], df_resultados_regulador['estoque_total'], label='Estoque Total (B_total)', marker='x')
plt.plot(df_resultados_regulador['tempo'], df_resultados_regulador['derivada_estoque'], label='Derivada do Estoque (dB_total/dt)', marker='^')

# Personalizar gráfico
plt.xlabel('Tempo')
plt.ylabel('Valores')
plt.title('Resultados do Regulador ao longo do Tempo')
#plt.legend()
plt.grid(True)

# Salvar o gráfico como PDF
plt.savefig('/content/drive/My Drive/graficos_REGULADOR_compilado_Linear.pdf', format='pdf')

# Mostrar o gráfico
plt.show()



In [ ]:
## SALVAR EM ÚNICO PDF

# Ajuste do valor de E[t=0]
E_t0 = sum(valores_iniciais_setores.values())
print(f"E[t=0]: {E_t0}")

def plot_graficos_variaveis_globais(model, nome_arquivo='/content/drive/My Drive/graficos_REGULADOR_individuais_Linear.pdf'):
    # Listas para armazenar dados para o gráfico
    tempos = []
    emissoes_agregadas = []
    estoque_total = []
    derivada_estoque_total = []

    # Preencher listas com dados das variáveis globais para cada tempo
    for t in model.time:
        if model.E[t].value is not None:  # Verificação para evitar valores None
            tempos.append(t)
            emissoes_agregadas.append(model.E[t].value)
            estoque_total.append(model.B_total_var[t].value)
            derivada_estoque_total.append(model.dB_total_dt[t].value)

    # Ajustar o valor de E[t=0] no gráfico
    emissoes_agregadas[0] = E_t0

    # Verificação se há múltiplos pontos no tempo
    if len(tempos) <= 1:
        print("Erro: O número de pontos no tempo é insuficiente para plotar o gráfico.")
        return

    # Verificação e impressão dos dados coletados
    print(f"Tempos: {tempos}")
    print(f"Emissões agregadas: {emissoes_agregadas}")

    # Criar uma figura e subplots para os gráficos das variáveis globais
    fig, ax = plt.subplots(3, 1, figsize=(10, 15))  # 3 linhas, 1 coluna para os gráficos

    # Gráfico de Emissões agregadas (E[t])
    ax[0].plot(tempos, emissoes_agregadas, marker='o', linestyle='-', color='blue', label='Emissões agregadas (E[t])')
    ax[0].set_title('Emissões agregadas ao longo do tempo (E[t])')
    ax[0].set_xlabel('Tempo')
    ax[0].set_ylabel('Emissões agregadas (E[t])')
    ax[0].grid(True)
    ax[0].legend()

    # Ajuste da escala do eixo Y para melhorar a visualização
    ax[0].set_ylim(min(emissoes_agregadas) * 0.9, max(emissoes_agregadas) * 1.1)
    ax[0].set_xticks(tempos)

    # Gráfico de Estoque total (B_total_var[t])
    ax[1].plot(tempos, estoque_total, marker='o', linestyle='-', color='green', label='Estoque total (B_total_var[t])')
    ax[1].set_title('Estoque total ao longo do tempo (B_total_var[t])')
    ax[1].set_xlabel('Tempo')
    ax[1].set_ylabel('Estoque total (B_total_var[t])')
    ax[1].grid(True)
    ax[1].legend()

    # Ajuste da escala do eixo Y para o gráfico de estoque total
    ax[1].set_ylim(min(estoque_total) * 0.9, max(estoque_total) * 1.1)
    ax[1].set_xticks(tempos)

    # Gráfico da Derivada do Estoque total (dB_total_dt[t])
    ax[2].plot(tempos, derivada_estoque_total, marker='o', linestyle='-', color='red', label='Derivada do estoque total (dB_total_dt[t])')
    ax[2].set_title('Derivada do estoque total ao longo do tempo (dB_total_dt[t])')
    ax[2].set_xlabel('Tempo')
    ax[2].set_ylabel('Derivada do estoque total (dB_total_dt[t])')
    ax[2].grid(True)
    ax[2].legend()

    # Ajustar layout, salvar o gráfico em PDF
    plt.tight_layout()

    # Salvar os gráficos no arquivo PDF
    with PdfPages(nome_arquivo) as pdf:
        pdf.savefig(fig)  # Salva a figura no PDF

    print(f'Gráficos salvos como {nome_arquivo}')

    # Exibir os gráficos
    plt.show()

# Chamar a função para plotar os gráficos e salvá-los em PDF
plot_graficos_variaveis_globais(model)


# Resultados

In [ ]:
# Verificação de compradores e vendedores líquidos de permissões
verificacao_permissoes = {}
for setor in model.setores:
    verificacao_permissoes[setor] = {}
    for t in model.time:
        # Verificar se as variáveis possuem valores numéricos
        if model.l_i[setor, t].value is None or pyo.value(model.l_i0_expr[setor, t]) is None:
            print(f"Erro: Variáveis l_i ou l_i0 não possuem valores definidos para setor {setor}, tempo {t}.")
            continue  # Pula para o próximo setor/tempo

        # Calcular y_i = l_i - l_i0
        l_i = model.l_i[setor, t].value
        l_i0 = pyo.value(model.l_i0_expr[setor, t])
        y_i = l_i - l_i0

        # Determinar se o setor é comprador ou vendedor
        if y_i > 0:
            status = 'Comprador'
        elif y_i < 0:
            status = 'Vendedor'
        else:
            status = 'Neutro'

        # Armazenar resultado
        verificacao_permissoes[setor][t] = {
            'l_i': l_i,
            'l_i0': l_i0,
            'y_i': y_i,
            'status': status
        }

# Exibir os resultados de compradores e vendedores
for setor, resultados_setor in verificacao_permissoes.items():
    for t, info in resultados_setor.items():
        print(f"Setor: {setor}, Tempo: {t}, l_i: {info['l_i']}, l_i0: {info['l_i0']}, y_i: {info['y_i']}, Status: {info['status']}")


In [ ]:
# Criar gráficos para cada setor
for setor, resultados_setor in verificacao_permissoes.items():
    tempos = list(resultados_setor.keys())
    y_i_values = [info['y_i'] for info in resultados_setor.values()]

    plt.figure(figsize=(8, 5))
    plt.plot(tempos, y_i_values, marker='o', linestyle='-', label=f'Setor {setor}')
    plt.axhline(0, color='gray', linestyle='--', linewidth=1)  # Linha de referência em y=0

    plt.xlabel("Tempo")
    plt.ylabel("y_i (l_i - l_i0)")
    plt.title(f"Evolução de y_i ao longo do tempo - Setor {setor}")
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
# Criar um dicionário com os dados de cap de cada setor
caps_setores = {
    "TranspCarga": [114937.896, 110758.336, 106578.776, 102399.216, 98219.657, 94040.097, 89860.537, 85680.977, 81501.416, 77321.857, 73142.297, 68962.738],
    "TransPass": [92843.424, 89467.299, 86091.175, 82715.050, 79338.926, 75962.801, 72586.677, 69210.552, 65834.427, 62458.303, 59082.179, 55706.054],
    "Saneamento": [85744.711, 82626.722, 79508.732, 76390.743, 73272.753, 70154.764, 67036.774, 63918.785, 60800.795, 57682.806, 54564.816, 51446.827],
    "FerroGussa": [51117.070, 49258.267, 47399.465, 45540.662, 43681.860, 41823.057, 39964.255, 38105.452, 36246.649, 34387.847, 32529.045, 30670.242],
    "Cimento": [34224.339, 32979.818, 31735.296, 30490.775, 29246.253, 28001.732, 26757.210, 25512.689, 24268.167, 23023.646, 21779.125, 20534.603],
    "PetroGas": [25821.432, 24882.471, 23943.510, 23004.549, 22065.587, 21126.626, 20187.665, 19248.704, 18309.743, 17370.782, 16431.820, 15492.859],
    "UTG": [12172.604, 11729.964, 11287.324, 10844.684, 10402.043, 9959.403, 9516.763, 9074.123, 8631.483, 8188.843, 7746.203, 7303.562],
    "ProdutosMet": [11361.545, 10948.398, 10535.251, 10122.104, 9708.957, 9295.810, 8882.662, 8469.515, 8056.368, 7643.221, 7230.074, 6816.927],
    "TransAere": [9549.379, 9202.129, 8854.879, 8507.629, 8160.378, 7813.128, 7465.878, 7118.628, 6771.378, 6424.128, 6076.878, 5729.627],
    "UTC": [7541.320, 7267.090, 6992.860, 6718.631, 6444.401, 6170.171, 5895.941, 5621.711, 5347.481, 5073.252, 4799.022, 4524.792],
    "Celulose": [5901.419, 5686.822, 5472.225, 5257.628, 5043.031, 4828.434, 4613.837, 4399.240, 4184.643, 3970.046, 3755.448, 3540.851],
    "VidCerm": [4561.417, 4395.547, 4229.678, 4063.808, 3897.938, 3732.068, 3566.199, 3400.329, 3234.459, 3068.590, 2902.720, 2736.850],
    "UTOD": [2425.250, 2337.059, 2248.868, 2160.677, 2072.486, 1984.295, 1896.105, 1807.914, 1719.723, 1631.532, 1543.341, 1455.150],
    "UTOC": [30.198, 29.100, 28.002, 26.904, 25.806, 24.707, 23.609, 22.511, 21.413, 20.315, 19.217, 18.119]
}


# Converter para DataFrame para fácil visualização
df_caps = pd.DataFrame(caps_setores)


In [ ]:
# Criar gráficos para cada setor
for setor, resultados_setor in verificacao_permissoes.items():
    tempos = list(resultados_setor.keys())
    y_i_values = [info['y_i'] for info in resultados_setor.values()]
    e_i_values = df_resultados_firmas[df_resultados_firmas['Setor'] == setor].set_index('Tempo')['Emissões (e_i)']

    # Obter os valores de cap do setor APENAS nos períodos correspondentes
    if setor in df_caps.columns:
        cap_tempos = list(range(len(df_caps)))  # Criando os períodos da base de dados (0 a 11)
        cap_values = df_caps[setor].values
    else:
        cap_tempos = []
        cap_values = []

    plt.figure(figsize=(8, 5))

    # Plotar y_i com cores diferentes para positivo e negativo
    for i in range(len(tempos) - 1):
        cor = 'green' if y_i_values[i] >= 0 else 'red'
        plt.plot(tempos[i:i+2], y_i_values[i:i+2], marker='o', linestyle='-', color=cor)

    # Plotar e_i como linha azul
    plt.plot(e_i_values.index, e_i_values.values, marker='s', linestyle='--', color='blue', label='Emissões (e_i)')

    # Plotar cap como linha roxa pontilhada APENAS nos períodos correspondentes
    if cap_tempos:
        plt.plot(cap_tempos, cap_values, marker='x', linestyle=':', color='purple', label='Cap (Permissões Iniciais)')

    plt.axhline(0, color='gray', linestyle='--', linewidth=1)  # Linha de referência em y=0

    plt.xlabel("Tempo")
    plt.ylabel("Valores")
    plt.title(f"Evolução de y_i, e_i e cap ao longo do tempo - Setor {setor}")

    # Adicionar legenda personalizada
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='green', marker='o', linestyle='-', label='y_i > 0 (Comprador)'),
        Line2D([0], [0], color='red', marker='o', linestyle='-', label='y_i < 0 (Vendedor)'),
        Line2D([0], [0], color='blue', marker='s', linestyle='--', label='Emissões (e_i)'),
        Line2D([0], [0], color='purple', marker='x', linestyle=':', label='Cap (Permissões Iniciais)')
    ]
    plt.legend(handles=legend_elements)

    plt.grid()
    plt.show()


In [ ]:
# Criar gráficos para cada setor
for setor, resultados_setor in verificacao_permissoes.items():
    tempos = list(resultados_setor.keys())
    y_i_values = np.array([info['y_i'] for info in resultados_setor.values()])

    plt.figure(figsize=(8, 5))

    # Identificar transições de sinal para mudar a cor da linha
    for i in range(len(tempos) - 1):
        cor = 'green' if y_i_values[i] >= 0 else 'red'
        plt.plot(tempos[i:i+2], y_i_values[i:i+2], marker='o', linestyle='-', color=cor)

    plt.axhline(0, color='gray', linestyle='--', linewidth=1)  # Linha de referência em y=0

    plt.xlabel("Tempo")
    plt.ylabel("y_i (l_i - l_i0)")
    plt.title(f"Evolução de y_i ao longo do tempo - Setor {setor}")
    plt.grid()
    plt.show()


In [ ]:
def plot_estoque_emissoes_por_setor(resultados):
    # Criar um arquivo PDF para salvar os gráficos
    pdf_filename = "/content/drive/My Drive/graficos_estoque_emissoes_por_setor.pdf"

    with PdfPages(pdf_filename) as pdf:
        # Iterar sobre os setores
        for setor, valores in resultados.items():
            tempos = []
            estoque = []
            emissoes = []

            # Coletar os dados
            for t, resultado in valores.items():
                tempos.append(t)
                estoque.append(resultado['B_i'])  # Estoque (B_i)
                emissoes.append(resultado['e_i'])  # Emissões (e_i)

            # Criar a figura
            plt.figure(figsize=(8, 5))

            # Plotar Estoque (B_i) em azul
            plt.plot(tempos, estoque, marker='o', linestyle='-', color='blue', label='Estoque (B_i)')

            # Plotar Emissões (e_i) em vermelho
            plt.plot(tempos, emissoes, marker='s', linestyle='--', color='red', label='Emissões (e_i)')

            # Configurações do gráfico
            plt.title(f'Estoque e Emissões ao longo do tempo - Setor: {setor}')
            plt.xlabel('Tempo')
            plt.ylabel('Valores')
            plt.legend()
            plt.grid(True)

            # Salvar no PDF
            pdf.savefig()
            plt.show()  # Mostrar o gráfico na tela
            plt.close()

    print(f"📄 Gráficos salvos em {pdf_filename}")

# Chamar a função para gerar os gráficos
plot_estoque_emissoes_por_setor(resultados_calculados)


# Gráficos dos Resultados setoriais

In [ ]:
# Criar gráficos para cada setor
for setor, resultados_setor in verificacao_permissoes.items():
    tempos = list(resultados_setor.keys())
    e_i_values = df_resultados_firmas[df_resultados_firmas['Setor'] == setor].set_index('Tempo')['Emissões (e_i)']

    # Obter os valores de cap do setor APENAS nos períodos correspondentes
    if setor in df_caps.columns:
        cap_tempos = list(range(len(df_caps)))  # Criando os períodos da base de dados (0 a 11)
        cap_values = df_caps[setor].values
    else:
        cap_tempos = []
        cap_values = []

    plt.figure(figsize=(8, 5))

    # Plotar e_i como linha azul tracejada com marcadores
    plt.plot(e_i_values.index, e_i_values.values, marker='s', linestyle='--', color='blue', label='Emissões (e_i)')

    # Plotar cap como linha preta sólida sem marcações
    if cap_tempos:
        plt.plot(cap_tempos, cap_values, linestyle='-', color='black', linewidth=1.5, label='Cap (Permissões Iniciais)')

    plt.axhline(0, color='gray', linestyle='--', linewidth=1)  # Linha de referência em y=0

    plt.xlabel("Tempo")
    plt.ylabel("Valores")
    plt.title(f"Evolução de Emissões (e_i) e Cap ao longo do tempo - Setor {setor}")

    # Adicionar legenda personalizada
    from matplotlib.lines import Line2D
    legend_elements = [
        Line2D([0], [0], color='blue', marker='s', linestyle='--', label='Emissões (e_i)'),
        Line2D([0], [0], color='black', linestyle='-', linewidth=1.5, label='Cap (Permissões Iniciais)')
    ]
    plt.legend(handles=legend_elements)

    plt.grid()
    plt.show()


In [ ]:
from matplotlib.lines import Line2D
import math

# Definir a cor azul da emissão com base na referência
color_blue_hex = "#1f77b4"

# Criar um mapeamento de anos (t=0 → 2024, t=10 → 2034)
anos = {t: 2024 + t for t in range(12)}

# Criar um dicionário de mapeamento para setores formatados
mapeamento_setores = {
    "TranspCarga": "Transporte de Carga",
    "TransPass": "Transporte de Passageiro",
    "Saneamento": "Saneamento",
    "FerroGussa": "Ferro-gusa, Aço e Ferroligas",
    "Cimento": "Cimento",
    "PetroGas": "Refino de Petróleo",
    "UTG": "Termoelétrica a Gás Natural",
    "ProdutosMet": "Produtos Metalúrgicos",
    "TransAere": "Transporte Aéreo",
    "UTC": "Termoelétrica a Carvão",
    "Celulose": "Celulose",
    "VidCerm": "Vidros e Cerâmicos",
    "UTOD": "Termoelétrica a Óleo Diesel",
    "UTOC": "Termoelétrica a Óleo Combustível"
}

# Criar o arquivo PDF para salvar os gráficos
pdf_filename = "/content/drive/My Drive/graficos_FIRMAS_Seção_Setores_do_resultado_Linear.pdf"

# Abrir o PDF para salvar os gráficos
with PdfPages(pdf_filename) as pdf:
    setores = list(verificacao_permissoes.keys())
    num_paginas = math.ceil(len(setores) / 3)  # Calcula quantas páginas serão necessárias

    for i in range(num_paginas):
        fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(8, 12))  # Criar 3 gráficos por página

        for j in range(3):  # Iterar sobre os 3 gráficos por página
            idx = i * 3 + j  # Índice do setor
            if idx >= len(setores):
                break  # Se não há mais setores, parar o loop

            setor = setores[idx]
            titulo_setor = mapeamento_setores.get(setor, setor)  # Obtém nome correto do setor

            resultados_setor = verificacao_permissoes[setor]
            tempos = list(resultados_setor.keys())
            e_i_values = df_resultados_firmas[df_resultados_firmas['Setor'] == setor].set_index('Tempo')['Emissões (e_i)']

            # Obter os valores de cap do setor APENAS nos períodos correspondentes
            if setor in df_caps.columns:
                cap_tempos = list(range(len(df_caps)))  # Criando os períodos da base de dados (0 a 11)
                cap_values = df_caps[setor].values
            else:
                cap_tempos = []
                cap_values = []

            ax = axes[j]  # Define qual eixo (gráfico) será usado

            # Plotar e_i como linha azul contínua sem marcadores
            ax.plot(e_i_values.index, e_i_values.values, linestyle='-', color=color_blue_hex, linewidth=2, label='Emissões (e_i)')

            # Plotar cap como linha preta sólida sem marcações
            if cap_tempos:
                ax.plot(cap_tempos, cap_values, linestyle='-', color='black', linewidth=1.5, label='Cap (Permissões Iniciais)')

            ax.axhline(0, color='gray', linestyle='--', linewidth=1)  # Linha de referência em y=0

            # Alterar os rótulos do eixo X para anos de 2 em 2
            anos_filtrados = {t: ano for t, ano in anos.items() if ano % 2 == 0}  # Filtrar apenas anos pares
            ax.set_xticks(list(anos_filtrados.keys()))
            ax.set_xticklabels(list(anos_filtrados.values()))

            ax.set_xlabel("Ano", fontsize=15)
            ax.set_ylabel("MtCO₂e", fontsize=15)
            ax.tick_params(axis='both', labelsize=15)

            ax.set_title(f'{titulo_setor}', fontsize=15)  # Agora o título muda corretamente

            # Criar a legenda personalizada
            legend_elements = [
                Line2D([0], [0], color=color_blue_hex, linestyle='-', linewidth=2, label='Emissões (e_i)'),
                Line2D([0], [0], color='black', linestyle='-', linewidth=1.5, label='Cap (Permissões Iniciais)')
            ]

            ax.legend().remove()  # Remover legenda de dentro do gráfico

            ax.grid()

        # Criar legenda abaixo dos gráficos
        fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, 0), ncol=2, frameon=False, fontsize=12)

        plt.tight_layout()  # Ajustar layout dos gráficos
        pdf.savefig(fig)  # Salvar no PDF
        plt.show() # Fechar figura para evitar excesso de gráficos na memória

print(f"📄 Gráficos salvos em {pdf_filename}")


In [ ]:
# Definir a cor azul da emissão com base na referência
color_blue_hex = "#1f77b4"

# Criar um mapeamento de anos (t=0 → 2024, t=10 → 2034)
anos = {t: 2024 + t for t in range(12)}

# Criar o arquivo PDF para salvar os gráficos
pdf_filename = "/content/drive/My Drive/graficos_FIRMAS_Seção_Setores_do_resultado_Linear_comprado.pdf"

# Abrir o PDF para salvar os gráficos
with PdfPages(pdf_filename) as pdf:
    setores = list(verificacao_permissoes.keys())
    num_paginas = math.ceil(len(setores) / 3)  # Calcula quantas páginas serão necessárias

    for i in range(num_paginas):
        fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(8, 12))  # Criar 3 gráficos por página

        for j in range(3):  # Iterar sobre os 3 gráficos por página
            idx = i * 3 + j  # Índice do setor
            if idx >= len(setores):
                break  # Se não há mais setores, parar o loop

            setor = setores[idx]
            resultados_setor = verificacao_permissoes[setor]

            tempos = np.array(list(resultados_setor.keys()))
            y_i_values = np.array([info['y_i'] for info in resultados_setor.values()])

            ax = axes[j]  # Define qual eixo (gráfico) será usado

            # Criar um array de cores correspondente ao sinal de y_i
            cores = np.where(y_i_values >= 0, 'green', '#FF6347')  # Verde para positivo, vermelho claro (tomato) para negativo

            # Plotar a linha contínua com mudanças de cor
            for k in range(len(tempos) - 1):
                ax.plot(tempos[k:k+2], y_i_values[k:k+2], linestyle='-', color=cores[k], linewidth=2)

            ax.axhline(0, color='gray', linestyle='--', linewidth=1)  # Linha de referência em y=0

            # Alterar os rótulos do eixo X para anos de 2 em 2
            anos_filtrados = {t: ano for t, ano in anos.items() if ano % 2 == 0}  # Filtrar apenas anos pares
            ax.set_xticks(list(anos_filtrados.keys()))
            ax.set_xticklabels(list(anos_filtrados.values()))

            ax.set_xlabel("Ano", fontsize=15)
            ax.set_ylabel("MtCO₂e", fontsize=15)
            ax.tick_params(axis='both', labelsize=15)

            ax.set_title(f'{setor}', fontsize=15)

            ax.grid()

        # Criar legenda abaixo dos gráficos
        fig.legend(
            handles=[
                Line2D([0], [0], color='green', linestyle='-', linewidth=2, label='Compra'),
                Line2D([0], [0], color='#FF6347', linestyle='-', linewidth=2, label='Venda')
            ],
            loc='upper center', bbox_to_anchor=(0.5, 0), ncol=2, frameon=False, fontsize=12
        )

        plt.tight_layout()  # Ajustar layout dos gráficos
        pdf.savefig(fig)  # Salvar no PDF
        plt.show() # Fechar figura para evitar excesso de gráficos na memória

print(f"📄 Gráficos salvos em {pdf_filename}")


# Preço

In [ ]:
# Função para formatar os resultados das firmas
def formatar_resultados_firmas(resultados_calculados):
    # Lista para armazenar os dados
    dados_tabela_firmas = []

    # Iterar pelos setores e tempos
    for setor, valores in resultados_calculados.items():
        for t, resultado in valores.items():
            # Adiciona os resultados das firmas em uma estrutura de tabela
            dados_tabela_firmas.append({
                'Setor': setor,
                'Tempo': t,
                'Preço (p)': resultado['p']
            })

    # Retorna os dados como DataFrame
    return pd.DataFrame(dados_tabela_firmas)

# Função para exibir e exportar os resultados filtrados
def exibir_resultados_firmas(df_resultados_firmas):
    # Filtrar apenas setor "TranspCarga"
    df_filtrado = df_resultados_firmas[df_resultados_firmas['Setor'] == 'TranspCarga'][['Tempo', 'Preço (p)']]

    # Exibir DataFrame filtrado
    display(df_filtrado)

    # Exportar para Excel
    nome_arquivo = '/content/drive/My Drive/resultados_filtrados_TranspCarga.xlsx'
    df_filtrado.to_excel(nome_arquivo, index=False)
    print(f"Resultados filtrados exportados para {nome_arquivo}")

# Formatar e exibir os resultados das firmas
df_resultados_firmas = formatar_resultados_firmas(resultados_calculados)
exibir_resultados_firmas(df_resultados_firmas)


In [ ]:
# Dados fornecidos
dados = {
    "Tempo": [0.00, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00, 2.25, 2.50, 2.75, 3.00, 3.25, 3.50, 3.75, 4.00, 4.25,
              4.50, 4.75, 5.00, 5.50, 6.00, 6.50, 7.00, 7.50, 8.00, 8.50, 9.00, 9.50, 10.00, 10.50, 11.00],
    "Preço (p)": [92.899626, 99.999990, 99.999996, 99.999995, 99.999995, 99.999995, 99.999995, 99.999995, 99.999994, 99.999994,
                  99.999994, 99.999994, 99.999994, 99.999993, 99.999993, 99.999993, 99.999988, 99.999984, 0.000103, 0.000103,
                  99.999995, 100.000000, 99.999998, 99.999997, 99.999994, 99.999993, 99.999872, 79.902886, 0.000107, 0.000107,
                  0.000108, 0.000108, 0.000100]
}

# Criar DataFrame
df = pd.DataFrame(dados)

# Criar o gráfico
plt.figure(figsize=(10, 5))
plt.plot(df["Tempo"], df["Preço (p)"], linestyle='-', color='blue', label="Preço (p)") #marker='o'

# Configurações do gráfico
plt.xlabel("Tempo", fontsize=14)
plt.ylabel("Preço (p)", fontsize=14)
plt.title("Evolução do Preço ao Longo do Tempo")
#plt.axhline(y=100, color='red', linestyle='--', label="Teto de 100 (se houver)")
plt.grid(True)
#plt.legend()

plt.tick_params(axis='x', labelsize=14)  # apenas eixo x
plt.tick_params(axis='y', labelsize=14)  # apenas eixo y

# Exibir o gráfico
plt.show()
